In [1]:
# Dataset

import os
from PIL import Image
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import numpy as np


class UCSDPed2Dataset(Dataset):
    def __init__(self, root_dir, sequence_length=5, image_size=128, mode="train"):
        self.sequence_length = sequence_length
        self.image_size = image_size
        self.mode = mode

        if mode == "train":
            self.video_dir = os.path.join(root_dir, "train")
        else:
            self.video_dir = os.path.join(root_dir, "test")

        self.transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor()
        ])

        self.samples = []
        self._prepare_samples()

    def _prepare_samples(self):
        video_folders = sorted(os.listdir(self.video_dir))

        for video in video_folders:
            video_path = os.path.join(self.video_dir, video)

            if not os.path.isdir(video_path):
                continue

            frames = sorted([
                f for f in os.listdir(video_path)
                if f.lower().endswith((".jpg", ".png", ".tif"))
            ])

            for i in range(len(frames) - self.sequence_length):
                self.samples.append((
                    video_path,
                    frames[i:i + self.sequence_length],
                    frames[i + self.sequence_length]
                ))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        video_path, input_frames, target_frame = self.samples[idx]
    
        input_tensor = []
        for f in input_frames:
            img = Image.open(os.path.join(video_path, f)).convert("L")
            img = self.transform(img)
            input_tensor.append(img)
    
        input_tensor = torch.stack(input_tensor, dim=0)
    
        target_img = Image.open(
            os.path.join(video_path, target_frame)
        ).convert("L")
        target_img = self.transform(target_img)
    
        video_name = os.path.basename(video_path)  # e.g., "Test001" or "Test002"
    
        return input_tensor, target_img, video_name, target_frame


In [2]:
# Model

import torch.nn as nn


class CNNEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, 3, 2, 1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, 2, 1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, 2, 1),
            nn.ReLU()
        )

    def forward(self, x):
        return self.encoder(x)


class ConvLSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.conv = nn.Conv2d(
            input_dim + hidden_dim,
            4 * hidden_dim,
            3,
            padding=1
        )

    def forward(self, x, h, c):
        combined = torch.cat([x, h], dim=1)
        i, f, o, g = torch.split(self.conv(combined), self.hidden_dim, dim=1)

        i = torch.sigmoid(i)
        f = torch.sigmoid(f)
        o = torch.sigmoid(o)
        g = torch.tanh(g)

        c = f * c + i * g
        h = o * torch.tanh(c)
        return h, c


class ConvLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.cell = ConvLSTMCell(input_dim, hidden_dim)

    def forward(self, x):
        B, T, C, H, W = x.size()
        h = torch.zeros(B, C, H, W, device=x.device)
        c = torch.zeros_like(h)

        for t in range(T):
            h, c = self.cell(x[:, t], h, c)

        return h


class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, 4, 2, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.decoder(x)

class ChannelAttention(nn.Module):
    def __init__(self, in_channels, reduction=8):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.mlp = nn.Sequential(
            nn.Conv2d(in_channels, in_channels // reduction, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_channels // reduction, in_channels, 1, bias=False)
        )

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.mlp(self.avg_pool(x))
        max_out = self.mlp(self.max_pool(x))
        return self.sigmoid(avg_out + max_out)

class SpatialAttention(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=7, padding=3, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        return self.sigmoid(self.conv(x))
        
class SpatioTemporalAttention(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.channel_att = ChannelAttention(channels)
        self.spatial_att = SpatialAttention()

    def forward(self, x):
        x = x * self.channel_att(x)
        x = x * self.spatial_att(x)
        return x

class FutureFramePredictorWithAttention(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = CNNEncoder()
        self.convlstm = ConvLSTM(128, 128)
        self.attention = SpatioTemporalAttention(128)
        self.decoder = Decoder()

    def forward(self, x):
        encoded = torch.stack(
            [self.encoder(x[:, t]) for t in range(x.size(1))],
            dim=1
        )

        h = self.convlstm(encoded)
        h = self.attention(h)   # 🔥 ATTENTION HERE
        return self.decoder(h)


In [3]:
import os
import numpy as np
import torch
import cv2
from tqdm import tqdm
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

test_dataset = UCSDPed2Dataset(
    root_dir="../datasets/UCSDped2",
    sequence_length=5,
    image_size=128,
    mode="test"
)

test_loader = DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False
)

model = FutureFramePredictorWithAttention().to(device)
model.load_state_dict(torch.load("attention_ucsd_ped2.pth", map_location=device))
model.eval()

# =========================
# HELPERS
# =========================
def normalize_01(x: np.ndarray):
    x = x.astype(np.float32)
    mn, mx = float(x.min()), float(x.max())
    return (x - mn) / (mx - mn + 1e-8)

def heatmap_to_boxes_connected_components(
    heat_01,         # HxW float in [0,1]
    thr=0.35,        # start low to ensure non-empty mask
    min_area=20,
    keep_top_k=1
):
    """
    Uses connected components instead of contours.
    More robust when contours are empty due to morphology or thin shapes.
    Returns: boxes(list), mask_u8, debug(dict)
    """
    heat_01 = heat_01.astype(np.float32)

    # Threshold directly in float space
    mask = (heat_01 >= thr).astype(np.uint8)  # 0/1

    selected = int(mask.sum())
    total = mask.size
    selected_ratio = selected / (total + 1e-8)

    if selected == 0:
        # return empty but with debug
        dbg = {
            "thr": float(thr),
            "heat_min": float(heat_01.min()),
            "heat_max": float(heat_01.max()),
            "heat_mean": float(heat_01.mean()),
            "selected_px": selected,
            "selected_ratio": float(selected_ratio),
            "components": 0,
            "boxes": 0
        }
        return [], (mask * 255).astype(np.uint8), dbg

    # Connected components on binary mask
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
    # stats: [label, x, y, w, h, area] but actually columns: [x,y,w,h,area]
    # label 0 is background; start from 1
    boxes = []
    for lab in range(1, num_labels):
        x, y, w, h, area = stats[lab]
        if area >= min_area:
            boxes.append((int(x), int(y), int(w), int(h), int(area)))

    # Sort by area desc
    boxes.sort(key=lambda b: b[4], reverse=True)

    # Keep top-k
    boxes = boxes[:keep_top_k]

    # Drop area from final output
    boxes_out = [(x, y, w, h) for (x, y, w, h, area) in boxes]

    dbg = {
        "thr": float(thr),
        "heat_min": float(heat_01.min()),
        "heat_max": float(heat_01.max()),
        "heat_mean": float(heat_01.mean()),
        "selected_px": selected,
        "selected_ratio": float(selected_ratio),
        "components": int(num_labels - 1),
        "boxes": len(boxes_out)
    }

    return boxes_out, (mask * 255).astype(np.uint8), dbg

# =========================
# PARAMETERS (DEBUG-FIRST)
# =========================
output_root = "boxed_videos"
os.makedirs(output_root, exist_ok=True)

fps = 10
thr = 0.35          # start low; increase later to make it stricter
min_area = 20
keep_top_k = 1      # draw only one red box

alpha = 0.3
beta = 0.7

def open_writer_for_video(video_name, frame_w, frame_h):
    out_path = os.path.join(output_root, f"{video_name}.mp4")
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    return cv2.VideoWriter(out_path, fourcc, fps, (frame_w, frame_h)), out_path

# =========================
# RUN + EXPORT
# =========================
writer = None
current_video_name = None

with torch.no_grad():
    for idx, (inputs, target, video_name, target_frame_name) in enumerate(tqdm(test_loader, desc="Export red-box videos")):
        inputs = inputs.to(device)
        target = target.to(device)

        # new test folder => new video
        if current_video_name != video_name[0]:
            if writer is not None:
                writer.release()
                writer = None

            current_video_name = video_name[0]
            writer, out_path = open_writer_for_video(current_video_name, 128, 128)
            print(f"\nWriting: {out_path}")

        pred = model(inputs)

        # scalar score (not used for drawing now)
        pixel_err = torch.mean((pred - target) ** 2)
        feat_pred = model.encoder(pred)
        feat_gt = model.encoder(target)
        feature_err = torch.mean((feat_pred - feat_gt) ** 2)
        score = alpha * pixel_err + beta * feature_err

        # localization heatmap from absolute error (NO percentile)
        err_map = (pred - target).abs()[0, 0].detach().cpu().numpy()
        heat_01 = normalize_01(err_map)

        boxes, mask_u8, dbg = heatmap_to_boxes_connected_components(
            heat_01,
            thr=thr,
            min_area=min_area,
            keep_top_k=keep_top_k
        )

        # Print diagnostics occasionally
        if idx % 200 == 0:
            print(f"[{current_video_name}] idx={idx} dbg={dbg} "
                  f"err(min={err_map.min():.6f}, max={err_map.max():.6f}, mean={err_map.mean():.6f}) "
                  f"score={score.item():.6f}")

        # draw on GT
        gt_u8 = (target[0, 0].detach().cpu().numpy() * 255.0).astype(np.uint8)
        final = cv2.cvtColor(gt_u8, cv2.COLOR_GRAY2BGR)

        # RED box
        for (x, y, w, h) in boxes:
            cv2.rectangle(final, (x, y), (x + w, y + h), (0, 0, 255), 2)

        # small debug text so you can confirm
        cv2.putText(final, f"{current_video_name} {target_frame_name[0]}", (6, 14),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(final, f"boxes={len(boxes)} thr={thr}", (6, 28),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1, cv2.LINE_AA)

        writer.write(final)

if writer is not None:
    writer.release()

print(f"\nDone. Videos saved in: {output_root}")

Device: cpu


Export red-box videos:   0%|                                                                  | 0/1950 [00:00<?, ?it/s]


Writing: boxed_videos\Test001.mp4


Export red-box videos:   0%|                                                          | 2/1950 [00:01<14:14,  2.28it/s]

[Test001] idx=0 dbg={'thr': 0.35, 'heat_min': 0.0, 'heat_max': 1.0, 'heat_mean': 0.03025859408080578, 'selected_px': 128, 'selected_ratio': 0.007812499999995231, 'components': 69, 'boxes': 0} err(min=0.000000, max=0.294098, mean=0.008899) score=0.000355


...


Writing: boxed_videos\Test012.mp4


Export red-box videos:  92%|██████████████████████████████████████████████████▊    | 1802/1950 [04:08<00:20,  7.25it/s]

[Test012] idx=1800 dbg={'thr': 0.35, 'heat_min': 0.0, 'heat_max': 1.0, 'heat_mean': 0.03408050537109375, 'selected_px': 188, 'selected_ratio': 0.011474609374992995, 'components': 77, 'boxes': 0} err(min=0.000001, max=0.257464, mean=0.008775) score=0.000357


Export red-box videos: 100%|███████████████████████████████████████████████████████| 1950/1950 [04:27<00:00,  7.28it/s]


Done. Videos saved in: boxed_videos
